**第5章 US Treasury and futures**

In [1]:
from myABBR import * ; import myUtil as mu

def makeUsTsyBond(effDT, matDT, cpnRT, faceAMT=100.0, settleDS=Tp1):
    # 発行,満期日 処理
    effDT, matDT = jDT(*effDT), jDT(*matDT)  
    # スケジュール, 債券オブジェクト
    bondSCD = ql.Schedule(effDT,matDT,pdFreqSA,calUSg,unADJ,unADJ,dtGENb,EoMt)
    bondOBJ = ql.FixedRateBond(settleDS, faceAMT, bondSCD, [cpnRT/100], dcAAb)
    return bondOBJ

# 発行(2022,9,30) 満期(2027,9,30) クーポン4.125
bondOBJ = makeUsTsyBond((2022,9,30), (2027,9,30), 4.125)
# CF近似値
cfPRC = bondOBJ.cleanPrice(6/100, dcAAb, cmpdCMP, freqSA, jDT(2023,7,6))
print(f'利回り6%, 7月6日受渡の価格: {cfPRC:.4f}')

利回り6%, 7月6日受渡の価格: 93.0700


In [2]:
# 利回り3.70%から価格計算
tradeDT,         bndYLD,   futPRC                         =\
jDT(2023,4,20), 3.7/100,  109+10/32       ; setEvDT(tradeDT)
settleDT = calUSg.advance(tradeDT,Tp1,DD)

accruAMT = bondOBJ.accruedAmount(settleDT)
cleanPRC = bondOBJ.cleanPrice(bndYLD, dcAAb, cmpdCMP, freqSA, settleDT)
display(pd.DataFrame({'settleDT':settleDT.ISO(), 'bondYield':bndYLD,
                'accruAMT':accruAMT, 'cleanPRC':cleanPRC }, index=[0]))
# InterestRateオブジェクト、キャッシュフロー表
iRateOBJ = ql.InterestRate(bndYLD, dcAAb, cmpdCMP, freqSA)
display(mu.bondCashFlow(bondOBJ, ir=iRateOBJ, past=1)[:3])

,settleDT,bondYield,accruAMT,cleanPRC
0,2023-04-21,0.037,0.23668,101.723815


,payDate,coupon,accruStart,accruEnd,amount,DF
0,2022-09-30,NaN,NaN,NaN,NaN,1.00000
1,2023-03-31,0.04125,2022-09-30,2023-03-31,2.0625,1.00000
2,2023-10-02,0.04125,2023-03-31,2023-09-30,2.0625,0.98484


In [3]:
# Act/Act Semiannualの経過利息の手計算
apr21 = dcAAb.dayCount(jDT(2023,3,31),jDT(2023,4,21))
sep30 = dcAAb.dayCount(jDT(2023,3,31),jDT(2023,9,30))
print(f'4月21日までの日数:{apr21}, 利払期間の日数:{sep30},',
      f'(hc)経過利息:{ 4.125/2 * apr21/sep30 :.5f}')

4月21日までの日数:21, 利払期間の日数:183, (hc)経過利息:0.23668


In [4]:
# 受渡適格:   発行日       満期日  クーポン  CF   市場利回り
dvBND  = [((2022,9,30), (2027,9,30), 4.125, 0.9305, 3.70),
          ((2022,8,31), (2027,8,31), 3.125, 0.8953, 3.69),
          ((2023,1,31), (2028,1,31), 3.500, 0.9011, 3.65)]
dvCF, dvYLD = list(zip(*dvBND))[3:]                     # cf,yld列の取り出し
bndOBJ = [makeUsTsyBond(iss, mat, cpn) for iss,mat,cpn,_,_ in dvBND]
iRToBJ = [ql.InterestRate(yld/100, dcAAb, cmpdCMP, freqSA) for yld in dvYLD]

dfGRS = pd.DataFrame()                                  #グロスベーシス用df
for bnd, iRT, cf, yld in zip(bndOBJ, iRToBJ, dvCF, dvYLD) :
    BPV    = ql.BondFunctions.basisPointValue(bnd, iRT, settleDT)
    clnPRC = bnd.cleanPrice(yld/100, dcAAb, cmpdCMP, freqSA, settleDT)
    dtyPRC = bnd.dirtyPrice(yld/100, dcAAb, cmpdCMP, freqSA, settleDT)
    gBASIS = clnPRC - futPRC*cf
    dfGRS  = pd.concat([dfGRS, pd.DataFrame({
            'coupon':bnd.nextCouponRate(), 'maturity':bnd.maturityDate().ISO(),
            'yield':yld, 'BPV':BPV, 'cleanPRC':clnPRC, 'dirtyPRC':dtyPRC,
            'CF':cf, 'gBASIS':gBASIS }, index=[0])], ignore_index=True)
    
print(f'先物価格: {futPRC:.6f}, 現物受渡日: {settleDT.ISO()}')
display(dfGRS.style.format(fmtFUT))

先物価格: 109.312500, 現物受渡日: 2023-04-21


,coupon,maturity,yield,BPV,cleanPRC,dirtyPRC,CF,gBASIS
0,4.1250%,2027-09-30,3.7000,-0.0410,101.7238,101.9605,0.93050,0.0085
1,3.1250%,2027-08-31,3.6900,-0.0394,97.7398,98.1814,0.89530,-0.1277
2,3.5000%,2028-01-31,3.6500,-0.0433,99.3430,100.1164,0.90110,0.8415


In [5]:
# レポ条件, ネットベーシス用df
repRT,       repEndDT,      dfNET        =\
5.10/100, jDT(2023,7,6), pd.DataFrame()

for bnd, cf, cPRC, dPRC, gBAS in \
            zip(bndOBJ, dvCF, dfGRS.cleanPRC, dfGRS.dirtyPRC, dfGRS.gBASIS) :
    # レポ年数
    repYR   = dcA360.yearFraction(settleDT, repEndDT)     #レポ年ベース
    # 利息収入, レポコスト, キャリー
    accStAMT= bnd.accruedAmount(settleDT)                 #レポstart経過利息
    accEdAMT= bnd.accruedAmount(repEndDT)                 #レポend経過利息
    cpnINC  = accEdAMT - accStAMT                         #クーポンincome    
    repCST  = repRT * repYR * dPRC                        #レポコスト
    carry   = cpnINC   - repCST                           #キャリー
    # ネットベーシス、フォワード、インプライドレポ
    nBASIS  = gBAS     - carry                            #ネットBasis    
    fwdPRC  = cPRC     - carry                            #フォワード価格
    impREP  = ((futPRC*cf+accEdAMT)/dPRC - 1)/repYR       #インプライドレポ
    # df作成
    dfNET = pd.concat([dfNET, pd.DataFrame({
           'accStAMT':accStAMT, 'accEdAMT':accEdAMT,'cpnINC':cpnINC,
           'dirtyPRC':dPRC,     'repCST'  :repCST,  'carry'  :carry,  
           'nBASIS'  :nBASIS,   'fwdPRC'  :fwdPRC,  'impREP' :impREP
                                          }, index=[0])],  ignore_index=True)
print(f'レポレート:{repRT:.4%}, レポエンド日:{repEndDT.ISO()},',
      f'レポ日数:{dcA360.dayCount(settleDT,repEndDT)}, レポ年数:{repYR:.4f}')
display(dfNET)

レポレート:5.1000%, レポエンド日:2023-07-06, レポ日数:76, レポ年数:0.2111


,accStAMT,accEdAMT,cpnINC,dirtyPRC,repCST,carry,nBASIS,fwdPRC,impREP
0,0.236680,1.093238,0.856557,101.960496,1.097775,-0.241217,0.249751,101.965033,0.039397
1,0.441576,1.086957,0.645380,98.181384,1.057086,-0.411706,0.284032,98.151513,0.037297
2,0.773481,1.508287,0.734807,100.116437,1.077920,-0.343114,1.184576,99.686070,-0.005046
